In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.8 MB 27.2 MB/s 
     |████████████████████████████████| 7.6 MB 54.8 MB/s 
     |████████████████████████████████| 182 kB 73.9 MB/s 


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import os
os.chdir('/content/gdrive/MyDrive')

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

In [ ]:
os.chdir('/content/gdrive/MyDrive/Colab Notebooks')

In [ ]:
!chmod 600 /root/.kaggle/kaggle.json
!kaggle datasets download -d jp797498e/twitter-entity-sentiment-analysis

In [ ]:
!unzip twitter-entity-sentiment-analysis

In [ ]:
import pandas as pd

In [ ]:
train = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/Reddit_Data.csv')

In [ ]:
train = train.dropna()

In [ ]:
train.tail()

,clean_comment,category
37244,jesus,0
37245,kya bhai pure saal chutiya banaya modi aur jab...,1
37246,downvote karna tha par upvote hogaya,0
37247,haha nice,1
37248,facebook itself now working bjp’ cell,0


In [ ]:
print(set(train['category'].values))

{0.0, 1.0, -1.0}


In [ ]:
from simpletransformers.classification import ClassificationModel

In [ ]:
bert = ClassificationModel('bert', 'bert-base-multilingual-uncased', num_labels=3, use_cuda=True, args={
    'reprocess_input_data': True,
    'use_cached_eval_features': False,
    'overwrite_output_dir': True,
    'num_train_epochs': 5,
    'silent': True
})
bert.train_model(train)

Downloading:   0%|          | 0.00/625 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/672M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-multilingual-uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model 

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/872k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.72M [00:00<?, ?B/s]

/usr/local/lib/python3.8/dist-packages/simpletransformers/classification/classification_model.py:612: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  warnings.warn(


(450, 1.1042613842752245)

In [ ]:
test = pd.read_csv('/content/gdrive/MyDrive/valid.csv')

In [ ]:
result, model_outputs, wrong_predictions = bert.eval_model(test)
bads = {}
for pred in wrong_predictions:
    if pred.label in bads:
        bads[pred.label] += 1
    else:
        bads[pred.label] = 1
print("wrong predictions")
print(str(len(wrong_predictions)) + ' wrong out of ' + str(len(test)))
bads

/usr/local/lib/python3.8/dist-packages/simpletransformers/classification/classification_model.py:1454: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  warnings.warn(


wrong predictions
118 wrong out of 180


{1: 65, 0: 53}

In [ ]:
model = ClassificationModel('bert', 'monsoon-nlp/hindi-tpu-electra', num_labels=3, use_cuda=True, args={
    'reprocess_input_data': True,
    'use_cached_eval_features': False,
    'overwrite_output_dir': True,
    'num_train_epochs': 4,
})
model.train_model(train)

Downloading:   0%|          | 0.00/736 [00:00<?, ?B/s]

You are using a model of type electra to instantiate a model of type bert. This is not supported for all configurations of models and can yield errors.


Downloading:   0%|          | 0.00/466M [00:00<?, ?B/s]

Some weights of the model checkpoint at monsoon-nlp/hindi-tpu-electra were not used when initializing BertForSequenceClassification: ['electra.encoder.layer.11.output.dense.weight', 'electra.encoder.layer.10.output.LayerNorm.bias', 'electra.encoder.layer.2.attention.self.value.weight', 'electra.encoder.layer.9.attention.output.dense.bias', 'electra.encoder.layer.8.attention.output.dense.bias', 'electra.encoder.layer.8.intermediate.dense.weight', 'electra.encoder.layer.5.attention.output.LayerNorm.weight', 'electra.embeddings.word_embeddings.weight', 'electra.encoder.layer.11.output.LayerNorm.weight', 'electra.encoder.layer.2.attention.output.dense.bias', 'electra.encoder.layer.1.attention.self.query.weight', 'electra.encoder.layer.1.output.dense.bias', 'electra.encoder.layer.1.attention.output.LayerNorm.bias', 'electra.encoder.layer.2.attention.self.query.weight', 'electra.encoder.layer.8.output.LayerNorm.weight', 'electra.encoder.layer.7.attention.output.dense.weight', 'electra.encode

Downloading:   0%|          | 0.00/181 [00:00<?, ?B/s]

...

Running Epoch 0 of 4:   0%|          | 0/90 [00:00<?, ?it/s]

Running Epoch 1 of 4:   0%|          | 0/90 [00:00<?, ?it/s]

Running Epoch 2 of 4:   0%|          | 0/90 [00:00<?, ?it/s]

Running Epoch 3 of 4:   0%|          | 0/90 [00:00<?, ?it/s]

(360, 1.124067829383744)

In [ ]:
result, model_outputs, wrong_predictions = model.eval_model(test)
bads = {}
for pred in wrong_predictions:
    if pred.label in bads:
        bads[pred.label] += 1
    else:
        bads[pred.label] = 1
print("wrong predictions")
print(str(len(wrong_predictions)) + ' wrong out of ' + str(len(test)))
bads

/usr/local/lib/python3.8/dist-packages/simpletransformers/classification/classification_model.py:1454: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  warnings.warn(


  0%|          | 0/180 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/23 [00:00<?, ?it/s]

wrong predictions
118 wrong out of 180


{1: 65, 0: 53}

In [ ]:
import torch
from transformers import AutoModelForSequenceClassification
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("cardiffnlp/twitter-roberta-base-sentiment")
model = AutoModelForSequenceClassification.from_pretrained("cardiffnlp/twitter-roberta-base-sentiment")

Downloading:   0%|          | 0.00/747 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/899k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/150 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/499M [00:00<?, ?B/s]

In [ ]:
inputs = tokenizer("i am walking happy", return_tensors="pt")
with torch.no_grad():
    logits = model(**inputs).logits

predicted_class_id = logits.argmax().item()
model.config.id2label[predicted_class_id]

'LABEL_2'

In [ ]:
from transformers import pipeline
sent_ana = pipeline(task='sentiment-analysis', model=model, tokenizer=tokenizer)

In [ ]:
sent = train.iloc[:,0].tolist()
label = train.iloc[:,1].tolist()

In [ ]:
pred = []
for i in sent:
  p = sent_ana(i)
  pred_lab = p[0]['label']
  pred.append(int(pred_lab[-1])-1)

In [ ]:
len(pred)

73

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(label[:len(pred)], pred)

0.4246575342465753

In [ ]:
print(list(zip(pred, label[:len(pred)])))

[(-1, -1.0), (-1, 0.0), (0, 1.0), (-1, 1.0), (0, 1.0), (0, 0.0), (0, 0.0), (0, 0.0), (0, 1.0), (-1, 1.0), (0, 1.0), (0, 1.0), (0, 1.0), (0, 0.0), (1, 0.0), (0, -1.0), (-1, 0.0), (0, 0.0), (0, 1.0), (0, -1.0), (-1, 1.0), (0, -1.0), (-1, 0.0), (0, -1.0), (-1, -1.0), (1, 1.0), (-1, 0.0), (0, 0.0), (0, 0.0), (0, 0.0), (1, 1.0), (0, 0.0), (-1, -1.0), (-1, 1.0), (-1, -1.0), (0, 0.0), (0, 1.0), (1, 1.0), (-1, -1.0), (0, -1.0), (1, 1.0), (0, 0.0), (-1, 0.0), (-1, 1.0), (-1, 1.0), (0, 1.0), (1, 1.0), (0, 0.0), (0, 1.0), (-1, -1.0), (0, 1.0), (-1, 0.0), (0, 0.0), (-1, -1.0), (0, 1.0), (0, -1.0), (-1, 0.0), (0, 0.0), (1, 0.0), (1, 0.0), (-1, -1.0), (-1, 0.0), (-1, -1.0), (1, 1.0), (1, 1.0), (-1, 1.0), (0, 1.0), (0, -1.0), (0, -1.0), (0, 1.0), (-1, 0.0), (0, 0.0), (-1, 1.0)]


In [ ]:
sent_ana(sent[2])

[{'label': 'LABEL_1', 'score': 0.7559412717819214}]

In [ ]:
!pip install googletrans==3.1.0a0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for googletrans: filename=googletrans-3.1.0a0-py3-none-any.whl size=16368 sha256=63888948be2985843c823e4a3973728024683b1eb2aa4b88b9a4ba8723044ea8
  Stored in directory: /root/.cache/pip/wheels/dd/59/af/8d6c96a719763990f1c548e36b17d9efdfb767f42f7ff39f53
Successfully built googletrans
  Attempting uninstall: googletrans
    Found existing installation: googletrans 3.0.0
    Uninstalling googletrans-3.0.0:
      Successfully uninstalled googletrans-3.0.0


In [ ]:
hindi_data = pd.read_csv('/content/gdrive/MyDrive/train.csv')

In [ ]:
hindi_data

,text,experience
0,चंद्रमोहन शर्मा को-प्रड्यूसर और लीड ऐक्टर अक्ष...,2
1,अगर आप इस फिल्म को देखने जा रहे हैं तो सबसे पह...,0
2,बॉलीवुड वाले चोरी-छिपे हॉलीवुड फिल्मों से कहान...,2
3,"बैनर :\nसंजय दत्त प्रोडक्शन्स प्रा.लि., रुपाली...",0
4,1959 में घटित चर्चित नानावटी कांड में एक क्राइ...,1
...,...,...
713,31 अक्टूबर 1984 को काला दिवस कहा जाता है। इस द...,1
714,\n\nगुंडे को देख सत्तर और अस्सी के दशक का सिने...,1
715,Chandermohan.sharma@timesgroup.com ग्लैमर इंडस...,2
716,"निर्माता :\nसुनीता गोवारीकर, अजय बिजली, संजीव ...",2


In [ ]:
!pip install googletrans==3.1.0a0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.1/55.1 KB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 69.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.4/133.4 KB 18.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 KB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.0/65.0 KB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 KB 6.4 MB/s eta 0:00:00
  Created wheel for googletrans: filename=googletrans-3.1.0a0-py3-none-any.whl size=16368 sha256=579003fd821717aac5c873f927ff0f0a24dc12d5ab6ee57ef70c05885c1ad561
  Stored in directory: /root/.cache/pip/wheels/dd/59/af/8d6c96a719763990f1c548e36b17d9efdfb767f42f7ff39f53
Successfully built googletrans
  Attempting uninstall: chardet
    Found existing installation: chardet 4.0.0
 

In [ ]:
from googletrans import Translator
translator = Translator()

In [ ]:
from textblob import TextBlob
import string
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
def boundaries(text):
  words = TextBlob(text).words
  out = []
  le = 0
  start_idx = 0
  for i in range(len(words)):
    src = translator.translate(words[i]).src
    if src == 'hi':
      if le == 0:
        start_idx = i
      le += 1
    elif words[i] in string.punctuation:
      if le>0:
        le += 1
    else:
      if start_idx+le-1 >= start_idx:
        out.append([start_idx, start_idx+le-1])
        le = 0
  return out

In [ ]:
boundaries(hindi_data.text[0])

[]

In [ ]:
print(translator.translate(hindi_data.text[0][0:77]))

Translated(src=hi, dest=en, text=If Chandramohan Sharma is to be believed by co-producer and lead actor Akshay Kumar, then his new film, pronunciation=None, extra_data="{'translat...")


In [ ]:
data_scrapped = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/posts_data.csv')

In [ ]:
comments = data_scrapped.comments.tolist()

In [ ]:
comments

['[\'Chris P Bacon may be the greatest dad joke of all time.\', \'There’s something magical about hearing people who are supposed to be emotionless and robotic laughing hysterically that just gets me. It’s always heartwarming to see people’s human side come out like this.\', \'The one guy pointing at the cat has some serious meme potential\', \'0:16 the man’s wheezing laugh in the first one almost sounds like the wheels of Chris P. Bacon just squeaking away 😂\', \'"it was born without hind legs" \\n\\nInsane laughing\', \'My god those 2 guys laughing over the model was just next level 😂😂😂\', \'Chris P. Bacon almost killed a man with laughter\', \'"Ugliest dog competit- oh my word, what happened to that thing?"\\n\\nThe genuine way that he read that KILLED me. How are you supposed to straight face your way through a segment when a picture like that comes up on screen? I would\\\'ve laughed just as hard as he did. That clip was hysterical\', \'Laughter IS the best medicine ever. \\nAlso 

In [ ]:
comments[-1][2:len(comments[-1])-2].split("""', \'""")

['Imagine him attending someone\'s funeral. 🤣🤣", \'his smile so contagious..ppl like him automatically brings smile',
 '2020- Zomato boy is smiling...\\n2021- Zomato boy is crying...',
 'Bhai pareshaan mat karo yaar usse. Kitna bhola  aur pyaara insaan hai',
 'I am smiling without a single joke in the video... This guy is a legend.',
 'Who came here after watching alakh sir 😂',
 'बस ज़िन्दगी में इस आदमी के जितना खुश होना है भाईसाब।\', "12 hours of work even in extreme wether and just 350 rupee of pay is pretty insane. I love and respect this guy and we should learn from him to be happy with whatever we have instead of whining over something we don\'t have.", \'Legends says he is still smiling 😂',
 'This smile is of an absolutely innocent, clear hearted, absolutely satisfied with his life.\\r\\nCarefree.... Mast maula ... Stay Blessed Buddy.']

In [ ]:
import pandas as pd

In [ ]:
import re
import pickle
with open('Emoji_Dict.p', 'rb') as fp:
    Emoji_Dict = pickle.load(fp)
Emoji_Dict = {v: k for k, v in Emoji_Dict.items()}

def convert_emojis_to_word(text):
    for emot in Emoji_Dict:
        text = re.sub(r'('+emot+')', "_".join(Emoji_Dict[emot].replace(",","").replace(":","").split()), text)
    return text
def remove_emojis(data):
    emoj = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U00002500-\U00002BEF"  # chinese char
        u"\U00002702-\U000027B0"
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        u"\U0001f926-\U0001f937"
        u"\U00010000-\U0010ffff"
        u"\u2640-\u2642" 
        u"\u2600-\u2B55"
        u"\u200d"
        u"\u23cf"
        u"\u23e9"
        u"\u231a"
        u"\ufe0f"  # dingbats
        u"\u3030"
                      "]+", re.UNICODE)
    return re.sub(emoj, '', data)

text = "game is on 🇦🇷"
remove_emojis(text)

'game is on '

In [ ]:
translator.translate('Bhai pareshaan mat karo yaar usse. Kitna bhola  aur pyaara insaan hai').src

'hi'

In [ ]:
Emoji_Dict

{'🥇': ':1st_place_medal:',
 '🥈': ':2nd_place_medal:',
 '🥉': ':3rd_place_medal:',
 '🆎': ':AB_button_(blood_type):',
 '🏧': ':ATM_sign:',
 '🅰': ':A_button_(blood_type):',
 '🇦 🇫': ':Afghanistan:',
 '🇦 🇱': ':Albania:',
 '🇩 🇿': ':Algeria:',
 '🇦 🇸': ':American_Samoa:',
 '🇦 🇩': ':Andorra:',
 '🇦 🇴': ':Angola:',
 '🇦 🇮': ':Anguilla:',
 '🇦 🇶': ':Antarctica:',
 '🇦 🇬': ':Antigua_&_Barbuda:',
 '♒': ':Aquarius:',
 '🇦 🇷': ':Argentina:',
 '♈': ':Aries:',
 '🇦 🇲': ':Armenia:',
 '🇦 🇼': ':Aruba:',
 '🇦 🇨': ':Ascension_Island:',
 '🇦 🇺': ':Australia:',
 '🇦 🇹': ':Austria:',
 '🇦 🇿': ':Azerbaijan:',
 '🔙': ':BACK_arrow:',
 '🅱': ':B_button_(blood_type):',
 '🇧 🇸': ':Bahamas:',
 '🇧 🇭': ':Bahrain:',
 '🇧 🇩': ':Bangladesh:',
 '🇧 🇧': ':Barbados:',
 '🇧 🇾': ':Belarus:',
 '🇧 🇪': ':Belgium:',
 '🇧 🇿': ':Belize:',
 '🇧 🇯': ':Benin:',
 '🇧 🇲': ':Bermuda:',
 '🇧 🇹': ':Bhutan:',
 '🇧 🇴': ':Bolivia:',
 '🇧 🇦': ':Bosnia_&_Herzegovina:',
 '🇧 🇼': ':Botswana:',
 '🇧 🇻': ':Bouvet_Island:',
 '🇧 🇷': ':Brazil:',
 '🇮 🇴': ':British_Indian_Ocean_T

In [ ]:
with open('Emoticon_Dict.p', 'rb') as fp:
    Emoticon_Dict = pickle.load(fp)

def convert_emoticons_to_word(text):
    for emot in Emoticon_Dict:
        text = re.sub(r'('+emot+')', "_".join(Emoticon_Dict[emot].replace(",","").replace(":","").split()), text)
    return text

text = "Hello :-) :-)"
convert_emoticons_to_word(text)

'Hello Happy_face_smiley Happy_face_smiley'

In [ ]:
Emoticon_Dict

{'Happy face or smiley': ':\\]',
 'Happy face smiley': '=\\)',
 'Laughing, big grin or laugh with glasses': 'B\\^D',
 'Very happy': ':-\\)\\)',
 'Frown, sad, andry or pouting': '>:\\(',
 'Crying': ":'\\(",
 'Tears of happiness': ":'\\)",
 'Horror': "D‑':",
 'Disgust': 'D:<',
 'Sadness': 'D:',
 'Great dismay': 'DX',
 'Surprise': ':o',
 'Shock': ':-0',
 'Yawn': '>:O',
 'Kiss': ':X',
 'Wink or smirk': ';D',
 'Tongue sticking out, cheeky, playful or blowing a raspberry': ':‑b',
 'Skeptical, annoyed, undecided, uneasy or hesitant': ':S',
 'Straight face': ':\\|',
 'Embarrassed or blushing': ':$',
 'Sealed lips or wearing braces or tongue-tied': ':&',
 'Angel, saint or innocent': '0;\\^\\)',
 'Evil or devilish': '>;\\)',
 'Cool': '\\|;‑\\)',
 'Bored': '\\|‑O',
 'Tongue-in-cheek': ':‑J',
 'Party all night': '#‑\\)',
 'Drunk or confused': '%\\)',
 'Being sick': ':###..',
 'Dump': '<:‑\\|',
 'Troubled': '\\(>_<\\)>',
 'Baby': "\\(';'\\)",
 'Nervous or Embarrassed or Troubled or Shy or Sweat dro

In [ ]:
def preprocess(text):
  emoji_rep = convert_emojis_to_word(text)
  emot_rep = convert_emoticons_to_word(emoji_rep)
  emoji_rem = remove_emojis(emot_rep)
  trans = translator.translate(emoji_rem)
  fin = re.sub(r'www.\S+', '', trans.text, flags=re.MULTILINE)
  if fin[0] == "'" or fin[0] == '"':
    fin = fin[1:]
  return fin

In [ ]:
emotio_dict = {'Happy':2, 'Angry':-1, 'Sad':-2, 'Neutral':0, 'Surprise':1, 'Joyful':3}
def make_dataframe(path, emotion, train):
  comments = []
  emos = []
  i=0
  data_scrapped = pd.read_csv(path)
  comment_blocks = data_scrapped.comments.tolist()
  for comment_block in comment_blocks:
    comment_list = comment_block[2:len(comment_block[0])-2].split("""', """)
    for comment in comment_list:
      temp = comment.split('", ')
      if len(temp)>1:
        for sub_comment in temp:
          if len(sub_comment)>0:
            x = preprocess(sub_comment)
            comments.append(x)
            emos.append(emotio_dict[emotion])
            print(i,'Done')
            i+=1
      else:
        if len(comment)>0:
          x = preprocess(comment)
          comments.append(x)
          emos.append(emotio_dict[emotion])
          print(i,'Done')
          i+=1
  dict = {'Comments': comments, 'Emotions': emos}
  Data = pd.DataFrame(dict)
  return Data

In [ ]:
happy_data = make_dataframe('/content/gdrive/MyDrive/Colab Notebooks/posts_data.csv', 'Happy')

0 Done
1 Done
2 Done
3 Done
4 Done
5 Done
6 Done
7 Done
8 Done
9 Done
10 Done
11 Done
12 Done
13 Done
14 Done
15 Done
16 Done
17 Done
18 Done
19 Done
20 Done
21 Done
22 Done
23 Done
24 Done
25 Done
26 Done
27 Done
28 Done
29 Done
30 Done
31 Done
32 Done
33 Done
34 Done
35 Done
36 Done
37 Done
38 Done
39 Done
40 Done
41 Done
42 Done
43 Done
44 Done
45 Done
46 Done
47 Done
48 Done
49 Done
50 Done
51 Done
52 Done
53 Done
54 Done
55 Done
56 Done
57 Done
58 Done
59 Done
60 Done
61 Done
62 Done
63 Done
64 Done
65 Done
66 Done
67 Done
68 Done
69 Done
70 Done
71 Done
72 Done
73 Done
74 Done
75 Done
76 Done
77 Done
78 Done
79 Done
80 Done
81 Done
82 Done
83 Done
84 Done
85 Done
86 Done
87 Done
88 Done
89 Done
90 Done
91 Done
92 Done
93 Done
94 Done
95 Done
96 Done
97 Done
98 Done
99 Done
100 Done
101 Done
102 Done
103 Done
104 Done
105 Done
106 Done
107 Done
108 Done
109 Done
110 Done
111 Done
112 Done
113 Done
114 Done
115 Done
116 Done
117 Done
118 Done
119 Done
120 Done
121 Done
122 Done
123

In [ ]:
happy_data

,Comments,Emotions
0,Chris P Bacon may be the greatest dad joke of ...,2
1,There’s something magical about hearing people...,2
2,The one guy pointing at the cat has some serio...,2
3,0:16 the man’s wheezing laugh in the first one...,2
4,"""it was born without hind legs"" \n\nInsane lau...",2
...,...,...
214,Who came here after watching alakh sir face_wi...,2
215,"Just want to be as happy as this man in life, ...",2
216,12 hours of work even in extreme wether and ju...,2
217,Legends says he is still smiling face_with_tea...,2


In [ ]:
sad_data = make_dataframe('/content/gdrive/MyDrive/posts_data.csv', 'Sad')

0 Done
1 Done
2 Done
3 Done
4 Done
5 Done
6 Done
7 Done
8 Done
9 Done
10 Done
11 Done
12 Done
13 Done
14 Done
15 Done
16 Done
17 Done
18 Done
19 Done
20 Done
21 Done
22 Done
23 Done
24 Done
25 Done
26 Done
27 Done
28 Done
29 Done
30 Done
31 Done
32 Done
33 Done
34 Done
35 Done
36 Done
37 Done
38 Done
39 Done
40 Done
41 Done
42 Done
43 Done
44 Done
45 Done
46 Done
47 Done
48 Done
49 Done


In [ ]:
sad_data

,Comments,Emotions
0,Hey Guys! Hope you like this #shortfilm #depre...,-2
1,"“People don’t fake depression, they fake being...",-2
2,Depression is when you can't explain yourself ...,-2
3,Literally.. Wish that every Parents could unde...,-2
4,Depression is not the feeling i want to die.\n...,-2
5,Crying without knowing any reason ....Is that ...,-2
6,Only those who have gone through this know the...,-2
7,Make a video on teenage depression where one i...,-2
8,The saddest truth is when you lie on the bed t...,-2
9,Feeling lucky that my parents identified my an...,-2
